In [2]:
from python_scrips.data import Income
from python_scrips.data import Compas
from python_scrips.data import Import
from python_scrips.dice import Dice
from python_scrips.models import MLP

import pandas as pd
import torch
import random
import numpy as np
from IPython.display import clear_output

## Load classifier, create data instance and dice instance 
#### See 'classifier_training.ipynb'

In [3]:
m = torch.load("trained_models/classifier_income_originaldataset.pt")
m.eval()

d = Income()
exp = Dice(d, m)
x = d.data_torch_test[4]
x_df = d.data_df_test.iloc[4]

ModuleNotFoundError: No module named 'Models'

## Example of an instance x and 5 counterfactuals

In [ ]:
cfs = exp.generate_cfs(x, distance_weight=0.5, diversity_weight=3, reg_weight=0.1, total_cfs=5, output='df')

In [ ]:
cfs[0]['age'] = cfs[0]['age'].astype('float').round().astype('int')
cfs[0]['priors_count'] = cfs[0]['priors_count'].astype('float').round().astype('int')
cfs[0]['recidivism'] = cfs[0]['recidivism'].astype('bool')

## Generate a lot of counterfactuals & create 'pairs'
#### See 'generate_a_lot_of_cfs.ipynb' 

In [ ]:
cfs = torch.load('cfs.pt')
x = d.data_torch_train
n_pairs = 9000*5

pairs = torch.zeros(n_pairs, exp.n_columns) # a pair is the difference between an instance x and its counterfactional cf
targets = torch.zeros(n_pairs) # a pair is either fair or unfair
pair_number = 0
for i in range(d.len_train):
    if i == 9000:
        break
    
    cfs[i] = d.arg_max(cfs[i])
    df = d.torch_to_df(cfs[i])
    
    for n_cf, cf in enumerate(cfs[i]):
        # create pairs
        difference = cf - x[i]
        pairs[pair_number] = difference
        
        # check (un)fairness
        if df['gender'][n_cf] == d.data_df_train['gender'][i] and df['race'][n_cf] == d.data_df_train['race'][i]:
            targets[pair_number] = 0
        else:
            targets[pair_number] = 1
        
        pair_number += 1

## Train fair netwerk

In [ ]:
# split = round(0.75 * n_pairs)
# x_train = pairs[:split].detach()
# x_test = pairs[split:].detach()
# y_train = targets[:split].detach()
# y_test = targets[split:].detach()

In [ ]:
# input_dim = len(x_train[0])
# hidden_dim = 100

# mlp_model = MLP(input_dim, hidden_dim)
# criterion = torch.nn.BCELoss() # BCE = binary cross entropy - our targets are binary 
# optimizer = torch.optim.Adam(mlp_model.parameters(), lr = 0.001)

# ### EVAL ###
# mlp_model.eval() # here sets the PyTorch module to evaluation mode. 
# y_train_hat = mlp_model(x_train)
# before_train = criterion(y_train_hat.squeeze(), y_train)
# print('Test loss before training' , before_train.item())

# ### TRAIN ###
# mlp_model.train() # here sets the PyTorch module to train mode. 
# tot_epoch = 401
# for epoch in range(tot_epoch):
#     optimizer.zero_grad()
#     # Forward pass
#     y_train_hat = mlp_model(x_train)
#     # Compute Loss
#     loss = criterion(y_train_hat.squeeze(), y_train)
    
#     if epoch%100==0:
#         y_test_hat = mlp_model(x_test)
#         print('Epoch: {} -- train loss: {} -- accuracy (test set): {}'.format(epoch, round(loss.item(), 3), mlp_model.accuracy(y_test_hat, y_test)))
#         y_test_hat = mlp_model(x_test)
        
#     # Backward pass
#     loss.backward()
#     optimizer.step()

# ### EVAL ###
# mlp_model.eval()
# y_test_hat = mlp_model(x_test)
# after_train = criterion(y_test_hat.squeeze(), y_test) 
# print('Test loss after Training' , after_train.item())

# print('The accuracy scrore on the test set:', mlp_model.accuracy(y_test_hat, y_test))

In [ ]:
# torch.save(mlp_model, "f_fair.pt")

## Find extra datapoints

In [ ]:
f_fair = torch.load("f_fair_income.pt")
x = d.data_torch_test[2]
cfs = exp.generate_cfs(x, f_fair=f_fair, total_cfs=3, distance_weight=0.5, diversity_weight=5, reg_weight=0.1, output='df')

In [ ]:
instance = d.data_torch_train[0]

cfs = exp.generate_cfs(instance, f_fair=f_fair, total_cfs=3, output='df')
    
random_int = random.randint(0, 2)

augmented = cfs[0].iloc[[random_int]].copy()

In [ ]:
augmented = pd.DataFrame(columns=d.column_names)
augmented['target'] = None

for n_instance in range(1, d.len_train):
    
    instance = d.data_torch_train[n_instance]
    
    cfs = exp.generate_cfs(instance, f_fair=f_fair, total_cfs=3, output='df')
    
    random_int = random.randint(0, 2)
    
    augmented.loc[n_instance] = cfs[0].iloc[random_int].copy()
    
    if n_instance % 10 == 0:
        
        augmented.to_csv('augmented.csv', index=False)

## Data augmentation

In [11]:
d = Income()
extra_data = pd.read_csv('output/income/augmented.csv')
original_data = d.data_df_train.iloc[0:7990,:]
target_original_data = d.target_df_train.iloc[0:7990]
original_data['target'] = target_original_data
augmented_dataset = pd.concat([original_data, extra_data])

<ipython-input-11-f62b390c4a38>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  original_data['target'] = target_original_data


## Train again

In [12]:
d_augmented = Import(data_df = augmented_dataset, cont_indices=[6, 7])
x_train = d_augmented.data_torch_train
x_test = d_augmented.data_torch_test
y_train = d_augmented.target_torch_train
y_test = d_augmented.target_torch_test

input_dim = len(x_train[0])
hidden_dim = 100

mlp_model = MLP(input_dim, hidden_dim)
criterion = torch.nn.BCELoss() # BCE = binary cross entropy - our targets are binary 
optimizer = torch.optim.Adam(mlp_model.parameters(), lr = 0.001)

### EVAL ###
mlp_model.eval() # here sets the PyTorch module to evaluation mode. 
y_train_hat = mlp_model(x_train)
before_train = criterion(y_train_hat.squeeze(), y_train)
print('Test loss before training' , before_train.item())

### TRAIN ###
mlp_model.train() # here sets the PyTorch module to train mode. 
tot_epoch = 501
for epoch in range(tot_epoch):
    optimizer.zero_grad()
    # Forward pass
    y_train_hat = mlp_model(x_train)
    # Compute Loss
    loss = criterion(y_train_hat.squeeze(), y_train)
    
    if epoch%100==0:
        y_test_hat = mlp_model(x_test)
        print('Epoch: {} -- train loss: {} -- accuracy (test set): {}'.format(epoch, round(loss.item(), 3), mlp_model.accuracy(y_test_hat, y_test)))
        y_test_hat = mlp_model(x_test)
        
    # Backward pass
    loss.backward()
    optimizer.step()

### EVAL ###
mlp_model.eval()
y_test_hat = mlp_model(x_test)
after_train = criterion(y_test_hat.squeeze(), y_test) 
print('Test loss after Training' , after_train.item())

print('The accuracy scrore on the test set:', mlp_model.accuracy(y_test_hat, y_test))

Test loss before training 0.7548929452896118
Epoch: 0 -- train loss: 0.755 -- accuracy (test set): 0.197
Epoch: 100 -- train loss: 0.323 -- accuracy (test set): 0.868
Epoch: 200 -- train loss: 0.264 -- accuracy (test set): 0.897
Epoch: 300 -- train loss: 0.251 -- accuracy (test set): 0.899
Epoch: 400 -- train loss: 0.242 -- accuracy (test set): 0.901
Epoch: 500 -- train loss: 0.237 -- accuracy (test set): 0.904
Test loss after Training 0.24146902561187744
The accuracy scrore on the test set: 0.904


In [33]:
torch.save(mlp_model, "mlp_income_augmented.pt")

## Generate CF's again

In [13]:
exp = Dice(d_augmented, mlp_model)

In [14]:
d_augmented.data_torch_test.shape[1]

29

In [15]:
n_instances = len(y_test)
all_instances = torch.zeros((n_instances, d_augmented.data_torch_test.shape[1]))
all_cfs = torch.zeros((n_instances, 5, d_augmented.data_torch_test.shape[1]))

In [16]:
for n_instance in range(d_augmented.len_test):
    
    clear_output(wait=True)
    print('instance ' + str(n_instance+1) + '/' + str(n_instances))
    print('percentage: ' + str(round(n_instance*100/n_instances, 2)) + '%')
    
    instance = d_augmented.data_torch_test[n_instance]
    
    cfs = exp.generate_cfs(instance, total_cfs=5)
    
    all_cfs[n_instance] = cfs[0].clone()
    
    if n_instance % 10 == 0:
        
        torch.save(all_cfs, "cfs_round2_income.pt")
        torch.save(all_instances, "instances_round2_income.pt")

instance 3995/3995
percentage: 99.97%


In [46]:
all_cfs

tensor([[[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2608, 0.3973],
         [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2595, 0.4017],
         [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2614, 0.3980],
         [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2519, 0.3948],
         [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.2659, 0.3958]],

        [[0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.4537, 0.3930],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4468, 0.3938],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4500, 0.4046],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4530, 0.3967],
         [0.0000, 0.0000, 1.0000,  ..., 0.0000, 0.4495, 0.3959]],

        [[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4212, 0.4966],
         [0.0000, 1.0000, 0.0000,  ..., 0.0000, 0.4220, 0.5020],
         [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4231, 0.5000],
         [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4443, 0.4989],
         [1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.4246, 0.5307]],

        ...,

     

In [48]:
d.data_df

,education,gender,marital_status,occupation,race,workclass,age,hours_per_week
0,School,Female,Widowed,Service,White,Self-Employed,62,66
1,School,Male,Single,Service,White,Private,18,25
2,Bachelors,Male,Single,Blue-Collar,White,Private,25,50
3,HS-grad,Male,Married,Professional,White,Private,33,40
4,School,Female,Single,Blue-Collar,White,Private,36,40
...,...,...,...,...,...,...,...,...
32556,Bachelors,Female,Single,White-Collar,Other,Government,25,40
32557,Bachelors,Male,Married,White-Collar,White,Private,32,45
32558,Bachelors,Male,Single,Blue-Collar,Other,Private,27,40
32559,HS-grad,Male,Married,Service,White,Government,59,40


In [61]:
d.data_df['workclass'].unique()

array(['Self-Employed', 'Private', 'Government', 'Other/Unknown'],
      dtype=object)

## Fairness metric

In [4]:
d = Income()
instances = torch.load("instances_round2_income.pt")
instances = d.torch_to_df(instances)
cfs = torch.load("cfs_round2_income.pt") # torch

In [5]:
n_instances = cfs.shape[0]
cfs_per_instance = cfs.shape[1]
fair_cfs = 0

for n_instance in range(n_instances):
    instance = instances.iloc[n_instance]
    cfs_i = d.torch_to_df(cfs[n_instance])
    for n_cf in range(cfs_per_instance):
        cf = cfs_i.iloc[n_cf]
        if instance['gender'] == cf['gender'] and instance['race'] == cf['race']:
            fair_cfs += 1
            
fair_ratio = fair_cfs / (n_instances * cfs_per_instance)  

In [6]:
fair_ratio

0.13541927409261578